In [17]:
begin
    using ModelingToolkit
    using AlgebraicPetri
    using AlgebraicPetri.Epidemiology
    using AlgebraicPetri.BilayerNetworks

    using Catlab, Catlab.Theories
    using Catlab.CategoricalAlgebra
    using Catlab.Graphics
    using Catlab.Graphics: Graphviz
    import Catlab.CategoricalAlgebra: migrate!
    using Catlab.WiringDiagrams
    using Catlab.Programs
    using Catlab.Programs.RelationalPrograms
    import Catlab.WiringDiagrams.DirectedWiringDiagrams: WiringDiagramACSet
    import Catlab.CategoricalAlgebra.CSets: parse_json_acset
    
    # using JSON

    using Random
    using DifferentialEquations
end

In [18]:
draw(d::WiringDiagram) = to_graphviz(d,
    orientation=LeftToRight,
    labels=true, label_attr=:xlabel,
    node_attrs=Graphviz.Attributes(
      :fontname => "Courier",
    ),
    edge_attrs=Graphviz.Attributes(
      :fontname => "Courier",
    )
)

draw (generic function with 1 method)

In [19]:
function MTKLoadLRN(f)
    return read_json_acset(LabelledReactionNet{Float64,Float64},f)
end

MTKLoadLRN (generic function with 1 method)

In [20]:
make_depvar(p,t) = :($p($t))
function MTKFormODEProb(lrxn::AbstractLabelledReactionNet,tspan)    
    r = lrxn[:rate]
    c = lrxn[:concentration]

    lpn = LabelledPetriNet(lrxn);
    bn = LabelledBilayerNetwork();
    migrate!(bn,lpn);
    
    varstmt = :(@variables t)
    @show varnames = bn[:variable]
    append!(varstmt.args, make_depvar.(bn[:variable], :t))
    
    paramstmt = :(@parameters)
    params = bn[:parameter]
    append!(paramstmt.args, bn[:parameter])
    
    diffstmt = :(D = Differential(t))
  
    ϕs = map(parts(bn, :Box)) do b
      vars = map(incident(bn, b,:call)) do i
        j = bn[i, :arg]
        return bn[j, :variable]
      end
      p = :(*($(bn[b, :parameter])))
      append!(p.args, vars)
      return :($(Symbol("ϕ$b")) = $p)
    end
  
    
    infs = map(parts(bn, :Qout)) do tv
      vars = map(incident(bn, tv, :infusion)) do wa
        j = bn[wa, :influx]
        return Symbol("ϕ$j")
      end
      p = :(+())
      append!(p.args, vars)
  
      # same for the outfluxes
      vars = map(incident(bn, tv, :effusion)) do wn
        j = bn[wn, :efflux]
        return :(- $(Symbol("ϕ$j")))
      end
      append!(p.args, vars)
      return p
    end
  
    zparts = zip(bn[:tanvar], infs)
  
    eqns = [:(D($tanvar) ~ $rhs) for (tanvar, rhs) in zparts]
    eq = :([])
    append!(eq.args, eqns)
    eqnstmt = :(eqs = $eq)
  
    varnameexpr = Expr(:tuple, varnames...)
    parnameexpr = Expr(:tuple, params...)

    eval(varstmt)
    eval(paramstmt)
    eval(diffstmt)
    map(eval,ϕs)
    eval(eqnstmt)
    sys = ODESystem(eqs, t, name=:PetriNet)
    prob = ODEProblem(sys, c, tspan, r)
    # sol = solve(prob,Tsit5())
  
    # return quote
    #   $varstmt
    #   $paramstmt
    #   $diffstmt
    #   $(ϕs...)
    #   $eqnstmt
    #   return $varnameexpr, $parnameexpr, ODEProblem(ODESystem(eqs, t, name=:PetriNet), $c, $tspan, $r)
    # end
    return prob
end

MTKFormODEProb (generic function with 1 method)

In [21]:
function MTKSolveODE(prob)
    return solve(prob,Tsit5())
end

MTKSolveODE (generic function with 1 method)

In [23]:
# Form Workflow presentation of FreeBiproductCategory
@present Workflow(FreeBiproductCategory) begin
    (File,LRN,TSpan,ODEProb,ODESol)::Ob 
    MTKLoadLRN::Hom(File,LRN)
    MTKFormODEProb::Hom(LRN⊗TSpan,ODEProb)
    MTKSolveODE::Hom(ODEProb,ODESol)
end

Presentation{ThBiproductCategory, Symbol}(Catlab.Theories.FreeBiproductCategory, (Ob = Catlab.Theories.FreeBiproductCategory.Ob{:generator}[File, LRN, TSpan, ODEProb, ODESol], Hom = Catlab.Theories.FreeBiproductCategory.Hom{:generator}[MTKLoadLRN, MTKFormODEProb, MTKSolveODE]), Dict(:MTKSolveODE => (:Hom => 3), :LRN => (:Ob => 2), :ODEProb => (:Ob => 4), :TSpan => (:Ob => 3), :File => (:Ob => 1), :ODESol => (:Ob => 5), :MTKLoadLRN => (:Hom => 1), :MTKFormODEProb => (:Hom => 2)), Pair[])

In [24]:
# Form wiring diagram of load_form_sim Workflow
load_form_sim = @program Workflow (f::File,ts::TSpan) begin # 
    lrn = MTKLoadLRN(f)
    ode_prob = MTKFormODEProb(lrn,ts)
    ode_sol = MTKSolveODE(ode_prob)
    return ode_sol 
end

WiringDiagram{ThBiproductCategory}([:File,:TSpan], [:ODESol], 
[ -2 => {inputs},
  -1 => {outputs},
  1 => Box(:MTKLoadLRN, [:File], [:LRN]),
  2 => Box(:MTKFormODEProb, [:LRN,:TSpan], [:ODEProb]),
  3 => Box(:MTKSolveODE, [:ODEProb], [:ODESol]) ],
[ Wire((-2,1) => (1,1)),
  Wire((-2,2) => (2,2)),
  Wire((1,1) => (2,1)),
  Wire((2,1) => (3,1)),
  Wire((3,1) => (-1,1)) ])

In [25]:
# Serialize program wiring diagram
# write_json_graph(load_form_sim,"diagram_load_form_sim.json") 
write_json_acset(load_form_sim.diagram, "forecasting_dwd.json")

857

In [26]:
# Example of reading in wiring diagram from JSON
# load_form_sim_roundtrip = read_json_graph(Symbol, Symbol, Nothing, "diagram_load_form_sim.json")
function parse_json_acset(::Type{T}, input::AbstractDict) where T <: ACSet # Catlab.CategoricalAlgebra.CSets.
  out = T()
  #=for (k,v) ∈ input
    add_parts!(out, Symbol(k), length(v))
  end
  for l ∈ values(input)
    for (i, j) ∈ enumerate(l)
      for k ∈ keys(j) # (k,v) = j # 
        v = j[k]
        vtype = eltype(out[Symbol(k)])
        if !(v isa vtype)
          v = vtype(v)
        end
        set_subpart!(out, i, Symbol(k), v)
      end
    end
  end
  out
  end=#
  for (k,v) ∈ input
    add_parts!(out, Symbol(k), length(v))
  end
  for l ∈ values(input)
    for (i, j) ∈ enumerate(l)
      for k ∈ keys(j) # (k, v) = j # 
        v = j[k]
        vtype = eltype(out[Symbol(k)])
        # if ((Symbol(k)==:box_type) | (Symbol(k)==:outer_in_port_type) | (Symbol(k)==:inner_in_port_type)) & (v isa String)
        if v isa String
            v = Meta.parse(v)
            if v isa Expr
                v = eval(v)
            end
        end
        if !(v isa vtype)
          v = vtype(v)
        end
        set_subpart!(out, i, Symbol(k), v)
      end
    end
  end
  out
end

parse_json_acset (generic function with 2 methods)

In [27]:
# Read in wiring diagram acset from file
rt_wd_acset = read_json_acset(WiringDiagramACSet{Any,Any,Any,DataType},"diagram_load_form_sim.json")
# rt_wd_acset = parse_json_acset(WiringDiagramACSet{Any,Any,Any,DataType},JSON.parsefile("diagram_load_form_sim.json"))

Box,value,box_type
1,MTKLoadLRN,Box{Symbol}
2,MTKFormODEProb,Box{Symbol}
3,MTKSolveODE,Box{Symbol}
InPort,in_port_box,in_port_type
1,1,File
2,2,LRN
3,2,TSpan
4,3,ODEProb
OutPort,out_port_box,out_port_type
1,1,LRN


In [28]:
# Check equality of read-in wd-acset to original
rt_wd_acset == load_form_sim.diagram

true

In [29]:
# Form roundtrip wiring diagram from read-in wd acset
load_form_sim_roundtrip = WiringDiagram{ThBiproductCategory,Any,Any,Any}(rt_wd_acset,nothing)

WiringDiagram{ThBiproductCategory}([:File,:TSpan], [:ODESol], 
[ -2 => {inputs},
  -1 => {outputs},
  1 => Box(:MTKLoadLRN, [:File], [:LRN]),
  2 => Box(:MTKFormODEProb, [:LRN,:TSpan], [:ODEProb]),
  3 => Box(:MTKSolveODE, [:ODEProb], [:ODESol]) ],
[ Wire((-2,1) => (1,1)),
  Wire((-2,2) => (2,2)),
  Wire((1,1) => (2,1)),
  Wire((2,1) => (3,1)),
  Wire((3,1) => (-1,1)) ])

In [30]:
# Check equality of roundtrip wiring diagram to original
load_form_sim == load_form_sim_roundtrip

true

In [31]:
# Visualize simulation plan
draw(load_form_sim_roundtrip)

Graph("G", true, "dot", Catlab.Graphics.Graphviz.Statement[Catlab.Graphics.Graphviz.Subgraph("", Catlab.Graphics.Graphviz.Statement[Catlab.Graphics.Graphviz.Node("n0in1", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:id => "in1")), Catlab.Graphics.Graphviz.Node("n0in2", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:id => "in2")), Catlab.Graphics.Graphviz.Edge(Catlab.Graphics.Graphviz.NodeID[Catlab.Graphics.Graphviz.NodeID("n0in1", "", ""), Catlab.Graphics.Graphviz.NodeID("n0in2", "", "")], OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}())], OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:rank => "source", :rankdir => "TB"), OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:style => "invis", :shape => "none", :label => "", :width => "0", :height => "0.333"), OrderedCollections.OrderedDict{Symbol, U

In [35]:
# Generate Julia function that executes simulation plan
mtk_hom_expr = to_hom_expr(FreeBiproductCategory,load_form_sim_roundtrip)
mtk_jfunc = Catlab.Programs.GenerateJuliaPrograms.compile(mtk_hom_expr)

function = (x1, x2;) -> begin
    begin
        v1 = (Main).MTKLoadLRN(x1)
        v2 = (Main).MTKFormODEProb(v1, x2)
        v3 = (Main).MTKSolveODE(v2)
        return v3
    end
end

In [36]:
# Expected output
#=  === mtk_jfunc == 
function = (x1, x2;) -> begin
    begin
        v1 = (Main).MTKLoadLRN(x1)
        v2 = (Main).MTKFormODEProb(v1, x2)
        v3 = (Main).MTKSolveODE(v2)
        return v3
    end
end
=#

In [37]:
# Apply generated function to lrxnet from MIRA integration demo
mtk_ode_sol = mtk_jfunc(joinpath(@__DIR__, ".", "lrxnet_Mira_TC_est.json"),(0,50))

varnames = bn[:variable] = [:S_City1, :E_City1, :I_City1, :A_City1, :SQ_City1, :H_City1, :R_City1, :EQ_City1, :D_City1, :S_City2, :E_City2, :I_City2, :A_City2, :SQ_City2, :H_City2, :R_City2, :EQ_City2, :D_City2]


retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 110-element Vector{Float64}:
  0.0
  7.970522410015634e-6
  8.767574651017196e-5
  0.0008847279875117352
  0.008855250397527367
  0.032263973511868105
  0.06980567061637252
  0.11988230539507619
  0.18383158473473754
  0.26358640233254044
  0.36208243599029427
  0.4808965281035089
  0.6202040888566542
  ⋮
 27.591378342734664
 28.841252138011964
 30.21526349928272
 31.735033026154802
 33.427483871703174
 35.32636432019689
 37.474136421486
 39.924147535656104
 42.742613428972554
 46.00787114934658
 49.801574346321964
 50.0
u: 110-element Vector{Vector{Float64}}:
 [999.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [998.9974146629728, 0.0003298627042708649, 0.9999968795283375, 2.00488908777486e-10, 0.0014625571331562442, 1.7420860490819836e-6, 9.894500934739281e-10, 0.000866078348361747, 1.0643056912836755e-6, 999.999926837436, 1.0357158273119205e-10, 3.13825600388478